# 1. Начало

In [3]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.4 MB

In [4]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''

In [5]:
!pip install openai

In [40]:
from openai import OpenAI

client = OpenAI(
  api_key=""
)

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  store=True,
  messages=[
    {"role": "user", "content": "Is Matvey smart?"}
  ]
)

print(completion.choices[0].message.content);

The intelligence of an individual named Matvey would depend on various factors, including their educational background, experiences, cognitive abilities, and skills. If you're referring to a specific Matvey, I would need more context to provide an accurate answer. Intelligence can also manifest in many different forms, such as emotional intelligence, analytical skills, creativity, and social skills.


In [41]:
os.environ['OPENAI_API_KEY'] = ""

In [42]:
!pip install sentence-transformers

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [7]:
!pip install -qU pypdf

# 2. Сырая версия

In [48]:
from langchain.embeddings import HuggingFaceEmbeddings
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from uuid import uuid4
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
import numpy as np

# loader = WebBaseLoader(
    # web_paths=("",)
# )
# docs = loader.load()
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    # chunk_size=300, 
    # chunk_overlap=50)
# splits = text_splitter.split_documents(docs)

docs = []
directory = '/kaggle/input/my-data/'
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)
splits = []
for file_path in os.listdir(directory):
    with open(directory+file_path, encoding='Windows-1251') as f:
        content = f.read()
        docs.append(content)
        f.close()
#splits.append(text_splitter.split_text(content))
#splits_new = np.array(splits).flatten()
#splits = list(splits_new)
#print(splits)
documents = []
for i in range(len(docs)):
    document = Document(
        page_content=docs[i],
        metadata={"source": "Internet"},
        id=i,
    )
    documents.append(document)

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents=documents, embedding=embedding)
# 
# document_1 = Document(
    # page_content="В Сингапуре проживает 5.6 миллионов человек",
    # metadata={"source": "tweet"},
    # id=1,
# )
#vectorstore.add_documents(documents=[document_1], ids=[str(uuid4())])

retriever = vectorstore.as_retriever()
# Prompt
prompt = PromptTemplate(
    input_variables=['page_content', 'question'],
    template=""" На основании следующего контекста ответь на вопрос:
    Контекст: {context}
    Вопрос: {question}
    Ответ:
    """
)

# LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
question = "Я инженер машинного обучения, хочу переехать в Европу. Куда посоветуешь поехать и почему?"
rag_chain.invoke(question)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


'Если вы инженер машинного обучения и хотите переехать в Европу, я бы рекомендовал рассмотреть следующие страны:\n\n1. **Германия**: Германия имеет стабильную экономику и высокий спрос на специалистов в области информационных технологий, включая машинное обучение. В стране активно развиваются стартапы и крупные компании, работающие в этой области. Кроме того, Германия предлагает программу профессиональной иммиграции для специалистов с профильным образованием, что может облегчить процесс получения визы.\n\n2. **Нидерланды**: В Нидерландах наблюдается рост числа рабочих мест в сфере ИТ, включая машинное обучение. Страна известна своим открытым подходом к иммиграции и высоким уровнем жизни. Заработная плата для программистов варьируется от 50 до 60 тысяч евро в год, что делает Нидерланды привлекательным местом для переезда.\n\n3. **Великобритания**: Лондон и Дублин являются важными центрами технологий в Европе. Великобритания предлагает конкурентоспособные зарплаты для специалистов в обла

In [49]:
results = vectorstore.similarity_search(
    Question,
    k=3,
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Сегодня мы расскажем о 10 лучших
странах для проживания программистов.
РЕЙТИНГ ЛУЧШИХ СТРАН ДЛЯ РАБОТЫ
ПРОГРАММИСТОМ
КИТАЙ
Население современного Китая почти
достигает цифры в 1,5 миллиарда
человек, поэтому совсем не
удивительно, что именно жители этого
государства составляют бОльшую часть
интернет пользователей мира. Рынок
различных гаджетов каждый год
пополняется новинками, но вот
ИТ-разработчики в Китае — главный
дефицит.
 город в Китае - одной из топовых стран
для переезда программиста 
Для переезда программисту необходимо
получить приглашение от будущего
работодателя и оформить разрешение на
работу по рабочей визе класса Z.
АВСТРАЛИЯ
В Австралии находятся
представительства таких крупных
IT-компаний как Facebook, Google, Amazon, Microsoft,
поэтому становится понятно, что и на
этом далеком материке можно найти
что-то интересное.
Для оформления визы и переезда, it
специалисту необходимо:
      подтвердить опыт работы;  
      сдать языковой экзамен IELTS;  
      пройти медобследов

In [50]:
docs = retriever.get_relevant_documents(question, k=2)
docs

[Document(metadata={'source': 'Internet'}, page_content='Сегодня мы расскажем о 10 лучших\nстранах для проживания программистов.\nРЕЙТИНГ ЛУЧШИХ СТРАН ДЛЯ РАБОТЫ\nПРОГРАММИСТОМ\nКИТАЙ\nНаселение современного Китая почти\nдостигает цифры в 1,5 миллиарда\nчеловек, поэтому совсем не\nудивительно, что именно жители этого\nгосударства составляют бОльшую часть\nинтернет пользователей мира. Рынок\nразличных гаджетов каждый год\nпополняется новинками, но вот\nИТ-разработчики в Китае — главный\nдефицит.\n город в Китае - одной из топовых стран\nдля переезда программиста \nДля переезда программисту необходимо\nполучить приглашение от будущего\nработодателя и оформить разрешение на\nработу по рабочей визе класса Z.\nАВСТРАЛИЯ\nВ Австралии находятся\nпредставительства таких крупных\nIT-компаний как Facebook, Google, Amazon, Microsoft,\nпоэтому становится понятно, что и на\nэтом далеком материке можно найти\nчто-то интересное.\nДля оформления визы и переезда, it\nспециалисту необходимо:\n      подт

# 3. Multi-query

In [52]:
from langchain.prompts import ChatPromptTemplate

template = """You are an AI language model assistant. Your task is to generate four
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [53]:
generate_queries.invoke({"question":question})

['1. Какой европейский город лучше всего подходит для инженеров машинного обучения, и какие факторы стоит учитывать при выборе места для переезда?',
 '',
 '2. Какие страны в Европе предлагают наилучшие возможности для карьерного роста в области машинного обучения, и почему именно они?',
 '',
 '3. Где в Европе сосредоточены ведущие компании и стартапы в сфере машинного обучения, и какие преимущества они предлагают для специалистов?',
 '',
 '4. Какие аспекты жизни и работы в Европе могут быть привлекательны для инженера машинного обучения, рассматривающего переезд?']

In [54]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]

retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

<ipython-input-54-9c4721758787>:7: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


3

In [55]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-NH30TT9NqHCsz62f6iFkxbuj on tokens per min (TPM): Limit 100000, Used 78223, Requested 38956. Please try again in 123h41m19.68s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

# 4. Fusion

In [18]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [20]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}
    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
docs

[(Document(metadata={'source': 'tweet'}, page_content='Не всем климат покажется комфортным. Тут довольно жарко, днем 30–33° C, ночью — 26–30° C при влажности 80–90 %'),
  0.06666666666666667),
 (Document(metadata={'source': 'tweet'}, page_content='Всё просто: вакансии искать на LinkedIn. Ищут много фронт- и бэкенда. Крайне популярны Java, Go, JavaScript, бывает интересное на C++, потому что есть много вакансий в области робототехники и HFT. Тут есть R&D у довольно большого количества разных проектов: много банков и HFT, есть Grab и GoJek — сильно более развитые аналоги Убера, имеют собственные платежные системы и кучу другого функционала, Lazada — местный Амазон, Ubisoft — которые пилят Assassin’s Creed, Acronis, Autodesk, ByteDance, Tencent, Motional, Bosch и куча компаний поменьше, ну и конечно, гора стартапов. Присутствует FAANG, но активно разработкой не занимается, хотя позиции иногда проскакивают.'),
  0.06557377049180328),
 (Document(metadata={'source': 'tweet'}, page_content='М

In [21]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Машина уровня Короллы обойдется в 120,000.'

# 5. Decomposition

In [23]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [26]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Chain
generate_queries_decomposition = (prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

question = 'Сколько стоит содержание машины в Сингапуре?'
questions = generate_queries_decomposition.invoke({"question":question})

In [27]:
questions

['1. Каковы основные расходы на содержание автомобиля в Сингапуре?',
 '2. Какие налоги и сборы необходимо учитывать при содержании машины в Сингапуре?',
 '3. Какова средняя стоимость топлива и обслуживания автомобиля в Сингапуре?']

In [28]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [29]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()


q_a_pairs = ""
for q in questions:
    
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [30]:
answer

'Средняя стоимость топлива и обслуживания автомобиля в Сингапуре может варьироваться, но можно выделить несколько ключевых аспектов:\n\n1. **Стоимость топлива**: Цены на топливо в Сингапуре обычно выше, чем в большинстве других стран. На момент последней информации, стоимость бензина может составлять около 2-2.5 SGD за литр, но это может изменяться в зависимости от рыночных условий.\n\n2. **Обслуживание автомобиля**: Регулярное техническое обслуживание и ремонт также требуют значительных затрат. В зависимости от типа автомобиля и необходимых услуг, расходы на обслуживание могут составлять от 500 до 1,500 SGD в год. Это включает в себя такие услуги, как замена масла, проверка тормозов, замена шин и другие профилактические работы.\n\nТаким образом, учитывая стоимость топлива и регулярные расходы на обслуживание, содержание автомобиля в Сингапуре может быть довольно дорогим, что делает его менее привлекательным по сравнению с использованием общественного транспорта.'

# 6. Individual answer


In [31]:
# Answer each sub-question individually 

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition / 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        
        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs, 
                                                                "question": sub_question})
        rag_results.append(answer)
    
    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

In [32]:
questions

['1. Каковы основные расходы на содержание автомобиля в Сингапуре?',
 '2. Какие налоги и сборы необходимо учитывать при содержании машины в Сингапуре?',
 '3. Какова средняя стоимость топлива и обслуживания автомобиля в Сингапуре?']

In [33]:
answers

['Основные расходы на содержание автомобиля в Сингапуре включают высокие цены на автомобили из-за заградительных пошлин, которые могут достигать 120 тысяч SGD для модели уровня Короллы. Дополнительно, стоимость парковки варьируется от 3 до 20 SGD в час, а месячный абонемент на парковку составляет около 150–200 SGD. Общественный транспорт в Сингапуре хорошо развит, что делает использование автомобиля менее необходимым.',
 'При содержании машины в Сингапуре необходимо учитывать высокие заградительные пошлины на автомобили, которые могут достигать 120,000 SGD для моделей уровня Короллы. Также следует учитывать стоимость парковки, которая варьируется от 3 до 20 SGD в час, или около 150-200 SGD за месячный абонемент. Эти расходы делают содержание автомобиля в Сингапуре довольно дорогим.',
 'Средняя стоимость автомобиля в Сингапуре, например, уровня Короллы, составляет около 120,000 SGD. Парковка в городе стоит от 3 до 20 SGD в час, а месячный абонемент на парковку — около 150-200 SGD. Общее

In [34]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

'Содержание машины в Сингапуре обходится довольно дорого. Основные расходы включают высокие заградительные пошлины на автомобили, которые могут достигать 120,000 SGD для моделей уровня Короллы. Кроме того, стоимость парковки варьируется от 3 до 20 SGD в час, а месячный абонемент на парковку составляет около 150-200 SGD. Эти факторы делают содержание автомобиля в Сингапуре финансово обременительным, особенно учитывая, что общественный транспорт в стране хорошо развит и может быть более экономичным вариантом.'

# 7. Step Back

In [35]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Elon Musk’s was born in what country?",
        "output": "what is Elon Musk’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [37]:
generate_queries_step_back = prompt | llm | StrOutputParser()
question = "Сколько стоит парковка машины в Сингапуре?"
generate_queries_step_back.invoke({"question": question})

'Каковы средние цены на парковку в крупных городах?'

In [39]:
# Response prompt 
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": question})

'Парковка машины в Сингапуре может стоить от 3 до 20 долларов в час, в зависимости от места. Месячный абонемент на парковку обойдется примерно в 150–200 долларов.'

# 9. HyDe

In [44]:
from langchain.prompts import ChatPromptTemplate

template = """Please write a source or paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_docs_for_retrieval = (
    prompt_hyde | llm | StrOutputParser() 
)

question = "Сколько стоит образование в Сингапуре?"
generate_docs_for_retrieval.invoke({"question":question})

'Образование в Сингапуре имеет различные ценовые категории в зависимости от уровня и типа учебного заведения. Например, стоимость обучения в государственных школах для местных студентов может варьироваться от 200 до 500 сингапурских долларов в год, в то время как для иностранных студентов эта сумма может увеличиваться до 5,000-10,000 сингапурских долларов. В частных школах и международных учебных заведениях цены могут значительно возрасти, достигая 20,000-40,000 сингапурских долларов в год. Высшее образование в университетах также имеет широкий диапазон цен: обучение в государственных университетах может стоить от 8,000 до 20,000 сингапурских долларов в год, тогда как частные университеты могут запрашивать от 20,000 до 50,000 сингапурских долларов. Важно учитывать, что эти суммы могут изменяться в зависимости от программы и специфики учебного заведения.'

In [45]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever 
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

[Document(metadata={'source': 'tweet'}, page_content='Машины очень дорогие из-за заградительных пошлин, и их содержание не менее дорогое. Машина уровня Короллы обойдется в 120К, её парковка в городе — ещё 3–20 долларов в час, а месячный абонемент на парковку будет около 150–200. Компенсируется тем, что ехать на машине банально некуда, общественный транспорт очень хороший, а такси дешёвое — 10–20 SGD.'),
 Document(metadata={'source': 'tweet'}, page_content='Медицина дорогая. Поэтому если у вас плохое здоровье, этот фактор нужно учитывать. Медицина хорошая, экстренная медицина — очень хорошая. Попасть на прием к GP гарантированно можно сегодня, к специалисту — обычно сегодня-завтра, в худшем случае — послезавтра. На планирование операции надо закладывать неделю и больше. А emergency — как и следует из названия — «вот прямо сейчас».'),
 Document(metadata={'source': 'tweet'}, page_content='Образование детей — мероприятие недешёвое. Это касается тех, кто планирует задержаться в стране надол

In [46]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retireved_docs,"question":question})

'Образование в Сингапуре, особенно в частных школах, стоит в среднем 30,000 SGD в год. Государственные школы в первую очередь доступны для детей граждан и постоянных резидентов, и конкуренция за места в них велика.'

# 10. Logical and Semantic routing